In [1]:
import pandas as pd
import fiona
import folium
import geopandas as gpd
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import math

In [2]:
Obesity_df = pd.read_csv("../data/3. Obesity-prevalence-by-state-2021.csv")
Diabeties_df = pd.read_csv("../data/US-CDI_diabetes_prevalence.csv")
df_fastfood = pd.read_csv("../data/fast_food.csv")
df_Pop = pd.read_csv("../data/Pop_dataUS.csv")
states_df = pd.read_csv("../data/statelatlong.csv")
states_df.rename(columns = {'City':'State'}, inplace = True)

In [3]:
Obesity_df.head()

,State,,Prevalence,95% CI,Latitude,Longitude,City
0,Alabama,AL,39.9,"(37.9, 41.8)",32.601011,-86.680736,Alabama
1,Alaska,AK,33.5,"(31.5, 35.5)",61.302501,-158.775020,Alaska
2,Arizona,AZ,31.3,"(30.0, 32.6)",34.168219,-111.930907,Arizona
3,Arkansas,AR,38.7,"(36.6, 40.8)",34.751928,-92.131378,Arkansas
4,California,CA,27.6,"(26.1, 29.1)",37.271875,-119.270415,California


In [4]:
Diabeties_df.head()

,Unnamed: 0,yearstart,locationabbr,locationdesc,question,datavalueunit,datavaluetype,datavalue
0,5841,2014,AK,Alaska,Prevalence of diagnosed diabetes among adults ...,%,Age-adjusted Prevalence,7.7
1,5927,2015,AK,Alaska,Prevalence of diagnosed diabetes among adults ...,%,Age-adjusted Prevalence,7.8
2,5869,2016,AK,Alaska,Prevalence of diagnosed diabetes among adults ...,%,Age-adjusted Prevalence,7.8
3,5926,2017,AK,Alaska,Prevalence of diagnosed diabetes among adults ...,%,Age-adjusted Prevalence,7.6
4,5856,2018,AK,Alaska,Prevalence of diagnosed diabetes among adults ...,%,Age-adjusted Prevalence,8.7


In [5]:
df_fastfood.head()

,address,latitude,longitude,name,province
0,800 N Canal Blvd,29.814697,-90.814742,SONIC Drive In,LA
1,800 N Canal Blvd,29.814697,-90.814742,SONIC Drive In,LA
2,206 Wears Valley Rd,35.803788,-83.580553,Taco Bell,TN
3,3652 Parkway,35.782339,-83.551408,Arby's,TN
4,2118 Mt Zion Parkway,33.562738,-84.321143,Steak 'n Shake,GA


In [6]:
df_Pop.head()

,SUMLEV,REGION,DIVISION,STATE,State_abbreviations,NAME,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,NPOPCHG_2020,...,RBIRTH2021,RDEATH2021,RNATURALINC2021,RINTERNATIONALMIG2021,RDOMESTICMIG2021,RNETMIG2021,Unnamed: 31,Latitude,Longitude,City
0,40,3,6,1,AL,Alabama,5024279,5024803,5039877,524,...,11.191613,12.890226,-1.698613,0.247201,4.398749,4.645950,NaN,32.601011,-86.680736,Alabama
1,40,4,9,2,AK,Alaska,733391,732441,732673,-950,...,12.667956,7.700425,4.967532,0.718033,-5.295151,-4.577118,NaN,61.302501,-158.775020,Alaska
2,40,4,8,4,AZ,Arizona,7151502,7177986,7276316,26484,...,10.584669,10.469547,0.115121,0.619608,12.871739,13.491347,NaN,34.168219,-111.930907,Arizona
3,40,3,7,5,AR,Arkansas,3011524,3012232,3025891,708,...,11.599962,12.671819,-1.071856,0.272932,5.304960,5.577892,NaN,34.751928,-92.131378,Arkansas
4,40,4,9,6,CA,California,39538223,39499738,39237836,-38485,...,10.778412,8.441637,2.336775,0.364223,-9.329701,-8.965478,NaN,37.271875,-119.270415,California


In [7]:
df_pop2 = df_Pop.iloc[:, [4,7,32,33,34]]
df_pop3 = df_pop2.dropna()
df_pop3


,State_abbreviations,POPESTIMATE2020,Latitude,Longitude,City
0,AL,5024803,32.601011,-86.680736,Alabama
1,AK,732441,61.302501,-158.775020,Alaska
2,AZ,7177986,34.168219,-111.930907,Arizona
3,AR,3012232,34.751928,-92.131378,Arkansas
4,CA,39499738,37.271875,-119.270415,California
5,CO,5784308,38.997934,-105.550567,Colorado
6,CT,3600260,41.518784,-72.757507,Connecticut
7,DE,991886,39.145251,-75.418921,Delaware
8,DC,690093,38.899349,-77.014567,District of Columbia
9,FL,21569932,27.975728,-83.833017,Florida


In [8]:
#creating the base map
#creating layer with population estimates on map
# Add a choropleth map to the base map
lats_longs_weight = list(map(list, zip(df_pop3["Latitude"],
                          df_pop3["Longitude"],
                          df_pop3["POPESTIMATE2020"]
                         )
               )
           )

m= folium.Map(location = [38.27312, -98.5821872], zoom_start = 4)
fg = folium.FeatureGroup(name="Population Estimates 2020")
fg.add_child(HeatMap(lats_longs_weight))

m.add_child(fg)
m

In [9]:
per_state = df_fastfood.groupby(['province', 'name']).count().max(level=0)
per_state = per_state.rename(columns={'index': 'name', 'name': 'count'})
per_state = df_fastfood.groupby(['province', 'name'])['name'].count().reset_index(name='count')
per_state_all = per_state.groupby(['province']).count().reset_index()
per_state1 = per_state.groupby(['province'])['count'].max().reset_index()
per_state2 = pd.merge(per_state1, per_state, on=['province', 'count'])
per_state2.head().set_index('province')
per_state_merge = pd.merge(per_state2, states_df, left_index=True, right_index=True)
per_state_merge.set_index('province')
per_state_merge

,province,count,name,State,Latitude,Longitude,State
0,AK,4,Subway,AL,32.601011,-86.680736,Alabama
1,AL,3,Taco Bell,AK,61.302501,-158.775020,Alaska
2,AR,18,McDonald's,AZ,34.168219,-111.930907,Arizona
3,AZ,58,McDonald's,AR,34.751928,-92.131378,Arkansas
4,CA,155,McDonald's,CA,37.271875,-119.270415,California
5,CO,27,Taco Bell,CO,38.997934,-105.550567,Colorado
6,CT,17,Subway,CT,41.518784,-72.757507,Connecticut
7,DE,11,Arby's,DE,39.145251,-75.418921,Delaware
8,FL,137,McDonald's,DC,38.899349,-77.014567,District of Columbia
9,GA,100,McDonald's,FL,27.975728,-83.833017,Florida


In [10]:
#count of states into map
for index, row in per_state_merge.iterrows():
    location = row.Latitude, row.Longitude
    folium.Marker(location, tooltip = (f'State Name:{row.State}\n {row.count}')).add_to(m)
m                 

In [12]:
m.add_child(folium.LayerControl())
# m.save('test.html')